In [19]:
#hide
#default_exp tools.copy_experiment
from nbdev.showdoc import *
from block_types.utils.nbdev_utils import nbdev_setup, TestRunner

nbdev_setup ()
tst = TestRunner (targets=['dummy'])

# Copy experiment indicated by argument

In [40]:
#export
import argparse
import sys
import inspect
import shutil
import joblib
import os
from distutils.dir_util import copy_tree

from hpsearch.experiment_manager import mypprint
from hpsearch.config.hpconfig import get_experiment_manager

In [41]:
#for tests
import pytest
import os
from hpsearch.examples.complex_dummy_experiment_manager import generate_data, init_em

## copy_experiment_contents

In [43]:
#export
def copy_experiment_contents (experiment=None, root=None, destination_folder='.', 
                              run_number=0, em_args=None):
    
    os.makedirs (destination_folder, exist_ok=True)
    
    em = get_experiment_manager ()
    
    # 3 write code about calling run_experiment
    root_path = em.get_path_experiments(folder=root)
    path_root_experiment = em.get_path_experiment (experiment, root_path=root_path)
    shutil.copy (f'{path_root_experiment}/parameters.pk', 
                 f'{destination_folder}/separate_parameters.pk')
    path_results = em.get_path_results (experiment, run_number, root_folder=root)
    copy_tree (f'{path_results}', destination_folder)

## copy_code

In [ ]:
#export
def copy_code (source_folder, destination_folder, destination_file=None,
               path_root_experiment=None):
    
    os.makedirs (destination_folder, exist_ok=True)
    destination_path = (f'{destination_folder}/best_experiment.py' if destination_file is None 
                        else f'{destination_folder}/{destination_file}')
    path_root_experiment = (source_folder if path_root_experiment is None
                            else path_root_experiment)
    path_results = source_folder
    fdest = open (destination_path, 'wt')
    
    # 1 write code from base manager
    base_manager = (
'''
class ExperimentManager ():
    def __init__ (**kwargs):
        pass
'''
    )
    fdest.write (base_manager)
    
    # 2 write code from subclassed manager
    em = get_experiment_manager ()
    source_path = inspect.getfile(em.__class__)
    fsrc = open (source_path, 'rt')
    original = fsrc.read ()
    fsrc.close ()
    original.replace ('from hpsearch.experiment_manager import ExperimentManager', '')
    fdest.write (original)
    
    # 3 write code about calling run_experiment
    parameters, other_parameters = joblib.load (f'{path_root_experiment}/parameters.pk')
    parameters = mypprint(parameters, dict_name='    parameters')
    run_experiment = (
f'''
def main (**kwargs):
    all_parameters = joblib.load (f{path_results}/parameters.pk)
{parameters}
    all_parameters.update (parameters)
    all_parameters.update (**kwargs)
    em = {em.__class__.__name__} ()
    em.run_experiment (all_parameters, path_results)

if __name__ == '__main__':
    main ()
'''
    )
    fdest.write (run_experiment)
    fdest.close ()

## copy_experiment_contents_and_code

In [ ]:
#export
def copy_experiment_contents_and_code (experiment=None, root=None, 
                                       content_dst_folder='.', 
                                       code_dst_folder='.',
                                       run_number=0, 
                                       destination_file=None, em_args=None):
    copy_experiment_contents (experiment=experiment, root=root, 
                              destination_folder=content_dst_folder, 
                              run_number=run_number, em_args=em_args)
    copy_code (content_dst_folder, code_dst_folder, destination_file=destination_file,
               path_root_experiment=content_dst_folder)

### Usage example

In [26]:
#export tests.tools.test_copy_experiment
def test_copy_content_and_code ():
    em = generate_data ('copy_experiment_contents')

    # function
    copy_experiment_contents_and_code (
        experiment=2, content_dst_folder='test_dest_folder_copy_exp_content', 
        code_dst_folder='test_my_code', run_number=0, destination_file='test_file.py'
    )

    assert sorted(os.listdir ('test_dest_folder_copy_exp_content'))==['dummy_experiment_manager.py',
     'logs.txt',
     'model_history.pk',
     'model_weights.pk',
     'parameters.json',
     'parameters.pk',
     'parameters.txt',
     'separate_parameters.pk',
     'summary.txt']
    
    # checks
    assert os.listdir ('test_my_code')==['test_file.py']
    f = open (f'test_my_code/test_file.py', 'rt')
    text = f.read()
    f.close()
    assert 'ExperimentManager ()' in text
    assert 'parameters=dict(epochs=5,' in text
    assert 'em = ComplexDummyExperimentManager ()' in text
    
    shutil.rmtree ('test_my_code')
    shutil.rmtree ('test_dest_folder_copy_exp_content')
    em.remove_previous_experiments ()

In [27]:
tst.run (test_copy_experiment, tag='dummy', do=False)

running test_copy_experiment


## copy_code_with_experiment_paths

In [25]:
#export
def copy_code_with_experiment_paths (experiment=None, root=None, destination_folder='.', destination_file=None,
                     run_number=0, em_args=None):
    em = get_experiment_manager ()
    root_path = em.get_path_experiments(folder=root)
    path_root_experiment = em.get_path_experiment (experiment, root_path=root_path)
    path_results = em.get_path_results (experiment, run_number, root_folder=root)
    copy_code (path_results, destination_folder, destination_file=destination_file,
               path_root_experiment=path_root_experiment)
    

### Usage example

In [26]:
#export tests.tools.test_copy_experiment
def test_copy_code_with_experiment_paths ():
    em = generate_data ('copy_code_with_experiment_paths')
    
    # function
    copy_experiment (experiment=2, destination_folder='test_dest_folder_copy_exp', 
                     destination_file='test_file.py', run_number=0)
    
    # checks
    assert os.listdir ('test_dest_folder_copy_exp')==['test_file.py']
    f = open (f'test_dest_folder_copy_exp/test_file.py', 'rt')
    text = f.read()
    f.close()
    assert 'ExperimentManager ()' in text
    assert 'parameters=dict(epochs=5,' in text
    assert 'em = ComplexDummyExperimentManager ()' in text
    
    em.remove_previous_experiments ()
    shutil.rmtree ('test_dest_folder_copy_exp')

In [27]:
tst.run (test_copy_code_with_experiment_paths, tag='dummy', do=False)

running test_copy_experiment


## parse_arguments_and_run

In [4]:
#export
def parse_args (args):
    parser = argparse.ArgumentParser(description='run experiment')
    parser.add_argument('-d', '--debug', action= "store_true")
    parser.add_argument('-e', '--experiments', type=int, nargs='+', default=None,  help="experiment numbers")
    parser.add_argument('--epochs', type=int, default=None,  help="number of epochs")
    parser.add_argument('-u', '--unfinished', action= "store_true")
    parser.add_argument('--runs', type=int, default=None,  help="number of runs")
    parser.add_argument('--root', type=str, default=None, help='name of root folder')
    parser.add_argument('-v', '--verbose', type=int, default=None, help='verbosity level: 0, 1, 2')
    pars = parser.parse_args(args)
    
    return pars

def parse_arguments_and_run (args, 
                             # for tests
                             em_args = None):
    
    pars = parse_args(args)

    other_parameters = dict(
                            use_process=not pars.debug,
                            root_folder=pars.root
                            )
    
    em = get_experiment_manager ()
    if pars.verbose is not None:
        em.set_verbose (pars.verbose)
    if em_args is not None:
        for k in em_args:
            setattr (em, k, em_args[k])
    
    if pars.epochs is not None:
        parameters = {em.name_epoch: int(pars.epochs)}
        other_parameters.update (prev_epoch=True)
        check_experiment_matches=False
    else:
        check_experiment_matches=True
        parameters = {}
    if pars.unfinished:
        other_parameters.update (check_finished=True, use_previous_best=False)
    
    
    
    em.rerun_experiment (experiments=pars.experiments, nruns=pars.runs, root_folder=pars.root, 
                         parameters=parameters, other_parameters=other_parameters, 
                         check_experiment_matches=check_experiment_matches)

def main():
    parse_arguments_and_run (sys.argv[1:])

### usage: performing more runs on previous experiments

In [5]:
#export tests.tools.test_rerun
def test_parse_arguments_and_run_more_runs ():
    em = generate_data ('parse_arguments_and_run_more_runs', other_parameters={'root_folder': 'newroot'})
    
    df = em.get_experiment_data (folder_experiments='newroot')
    assert df.shape==(9,25)

    args = ['-e', '4', '3', '--root', 'newroot', '--verbose', '1']
    parse_arguments_and_run (args)
    em.raise_error_if_run=True
    df = em.get_experiment_data (folder_experiments='newroot')
    x=[f'{i}_validation_accuracy' for i in range(5)]; assert df.columns.isin(x).sum()==5
    assert df.shape==(9,25)

    args = ['-e', '4', '3', '--runs', '10', '--root', 'newroot']
    em.raise_error_if_run=False
    parse_arguments_and_run (args)
    df = em.get_experiment_data (folder_experiments='newroot')
    assert df.shape==(9,45)
    x=[f'{i}_validation_accuracy' for i in range(10)]; assert df.columns.isin(x).sum()==10
    
    em.remove_previous_experiments ()

In [6]:
tst.run (test_parse_arguments_and_run_more_runs, tag='dummy', debug=False)

running test_parse_arguments_and_run_more_runs


requiring experiment number to be 4
running experiment 4 with parameters:
{'epochs': 15, 'offset': 0.3, 'rate': 0.03, 'noise': 0.1}
other_parameters:
{'verbose': 0, 'root_folder': 'newroot', 'log_message': 'fixed rate, multiple epochs values', 'script_path': '/home/jcidatascience/jaume/workspace/remote/hpsearch/hpsearch/examples/dummy_experiment_manager.py', 'lineno': 211, 'experiment_number': 4, 'suffix_results': '_validation_accuracy', 'path_results_big': 'test_parse_arguments_and_run_more_runs/newroot/experiments/00004/4', 'git_hash': "b'd479ff5e0ff0f2c4456dd529029e242a767e4eae\\n'", 'use_process': True}
experiment script: /tmp/ipykernel_37461/811600117.py, line: 45
doing run 0 out of 1

found completed: experiment number: 4, run number: 0 - score: 0.816651
{'epochs': 15, 'offset': 0.3, 'rate': 0.03, 'noise': 0.1}
skipping...
mean : 0.8166512271516049, std: 0.0
requiring experiment number to be 3
running experiment 3 with parameters:
{'epochs': 15, 'offset': 0.1, 'rate': 0.03, 'nois

### Increasing the number of epochs used in previous experiments

In [7]:
#export tests.tools.test_rerun
def test_parse_arguments_and_run_more_epochs ():
    em = init_em ('parse_arguments_and_run_more_epochs')

    # get reference result
    _ = em.create_experiment_and_run (parameters={'offset':0.1, 'rate': 0.05, 'epochs': 7})
    df = em.get_experiment_data ()
    display (df)
    em.remove_previous_experiments()

    # first 3 experiments
    _ = em.create_experiment_and_run (parameters={'offset':0.1, 'rate': 0.05, 'epochs': 5})
    _ = em.create_experiment_and_run (parameters={'offset':0.05, 'rate': 0.03, 'epochs': 6})
    _ = em.create_experiment_and_run (parameters={'offset':0.1, 'rate': 0.05, 'epochs': 9})
    _ = em.create_experiment_and_run (parameters={'offset':0.05, 'rate': 0.03, 'epochs': 10})
    _ = em.create_experiment_and_run (parameters={'offset':0.1, 'rate': 0.05, 'epochs': 11})
    df = em.get_experiment_data ()
    display (df)
    assert df.shape==(5,8)

    # more epochs
    #args = ['-e', '4', '3', '--epochs', '7', '-d']
    args = ['-e', '3', '--epochs', '7', '-d']
    parse_arguments_and_run (
        args, 
        em_args={'desired_path_results_previous_experiment':'test_parse_arguments_and_run_more_epochs/experiments/00001/0',
                 'desired_epochs': 1, 'desired_current_epoch': 7}
    )

    df = em.get_experiment_data ()
    print (df.shape)
    assert df.shape==(6,8)
    
    args = ['-e', '4', '--epochs', '7', '-d']
    parse_arguments_and_run (
        args, 
        em_args={'desired_path_results_previous_experiment':'test_parse_arguments_and_run_more_epochs/experiments/00000/0',
                 'desired_epochs': 2, 'desired_current_epoch': 7}
    )

    df = em.get_experiment_data ()
    print (df.shape)
    assert df.shape==(7,8)
    
    # *****************************************
    # *****************************************
    em.remove_previous_experiments ()
    em.desired_path_results_previous_experiment, em.desired_epochs, em.desired_current_epoch = None, None, None
    # first 3 experiments
    _ = em.create_experiment_and_run (parameters={'offset':0.1, 'rate': 0.05, 'epochs': 5})
    _ = em.create_experiment_and_run (parameters={'offset':0.05, 'rate': 0.03, 'epochs': 6})
    _ = em.create_experiment_and_run (parameters={'offset':0.1, 'rate': 0.05, 'epochs': 9})
    _ = em.create_experiment_and_run (parameters={'offset':0.05, 'rate': 0.03, 'epochs': 10})
    _ = em.create_experiment_and_run (parameters={'offset':0.1, 'rate': 0.05, 'epochs': 11})
    
    args = ['-e', '4', '3', '--epochs', '7', '-d']
    parse_arguments_and_run (args)
    df = em.get_experiment_data ()
    print (df.shape)
    assert df.shape==(7,8)
    
    em.remove_previous_experiments ()

In [8]:
tst.run (test_parse_arguments_and_run_more_epochs, tag='dummy')

running test_parse_arguments_and_run_more_epochs
fitting model with 7 epochs
epoch 0: accuracy: 0.15000000000000002
epoch 1: accuracy: 0.2
epoch 2: accuracy: 0.25
epoch 3: accuracy: 0.3
epoch 4: accuracy: 0.35
epoch 5: accuracy: 0.39999999999999997
epoch 6: accuracy: 0.44999999999999996


,offset,rate,epochs,0_validation_accuracy,0_test_accuracy,time_0,date,0_finished
0,0.1,0.05,7.0,0.45,0.55,0.001663,14:02:35.839428,True


fitting model with 5 epochs
epoch 0: accuracy: 0.15000000000000002
epoch 1: accuracy: 0.2
epoch 2: accuracy: 0.25
epoch 3: accuracy: 0.3
epoch 4: accuracy: 0.35
fitting model with 6 epochs
epoch 0: accuracy: 0.08
epoch 1: accuracy: 0.11
epoch 2: accuracy: 0.14
epoch 3: accuracy: 0.17
epoch 4: accuracy: 0.2
epoch 5: accuracy: 0.23
fitting model with 9 epochs
epoch 0: accuracy: 0.15000000000000002
epoch 1: accuracy: 0.2
epoch 2: accuracy: 0.25
epoch 3: accuracy: 0.3
epoch 4: accuracy: 0.35
epoch 5: accuracy: 0.39999999999999997
epoch 6: accuracy: 0.44999999999999996
epoch 7: accuracy: 0.49999999999999994
epoch 8: accuracy: 0.5499999999999999
fitting model with 10 epochs
epoch 0: accuracy: 0.08
epoch 1: accuracy: 0.11
epoch 2: accuracy: 0.14
epoch 3: accuracy: 0.17
epoch 4: accuracy: 0.2
epoch 5: accuracy: 0.23
epoch 6: accuracy: 0.26
epoch 7: accuracy: 0.29000000000000004
epoch 8: accuracy: 0.32000000000000006
epoch 9: accuracy: 0.3500000000000001
fitting model with 11 epochs
epoch 0: ac

,offset,rate,epochs,0_validation_accuracy,0_test_accuracy,time_0,date,0_finished
0,0.10,0.05,5.0,0.35,0.45,0.001308,14:02:35.881551,True
1,0.05,0.03,6.0,0.23,0.33,0.001343,14:02:35.908114,True
2,0.10,0.05,9.0,0.55,0.65,0.001755,14:02:35.938867,True
3,0.05,0.03,NaN,0.35,0.25,0.002603,14:02:35.971866,True
4,0.10,0.05,11.0,0.65,0.55,0.002215,14:02:36.007275,True


reading model from test_parse_arguments_and_run_more_epochs/experiments/00001/0/model_weights.pk
fitting model with 1 epochs
epoch 0: accuracy: 0.26
(6, 8)
reading model from test_parse_arguments_and_run_more_epochs/experiments/00000/0/model_weights.pk
fitting model with 2 epochs
epoch 0: accuracy: 0.39999999999999997
epoch 1: accuracy: 0.44999999999999996
(7, 8)
fitting model with 5 epochs
epoch 0: accuracy: 0.15000000000000002
epoch 1: accuracy: 0.2
epoch 2: accuracy: 0.25
epoch 3: accuracy: 0.3
epoch 4: accuracy: 0.35
fitting model with 6 epochs
epoch 0: accuracy: 0.08
epoch 1: accuracy: 0.11
epoch 2: accuracy: 0.14
epoch 3: accuracy: 0.17
epoch 4: accuracy: 0.2
epoch 5: accuracy: 0.23
fitting model with 9 epochs
epoch 0: accuracy: 0.15000000000000002
epoch 1: accuracy: 0.2
epoch 2: accuracy: 0.25
epoch 3: accuracy: 0.3
epoch 4: accuracy: 0.35
epoch 5: accuracy: 0.39999999999999997
epoch 6: accuracy: 0.44999999999999996
epoch 7: accuracy: 0.49999999999999994
epoch 8: accuracy: 0.549